In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy
import matplotlib.path as mpath
from functions import compute_ivt,to_nc
from matplotlib import rc,animation
from matplotlib.animation import FuncAnimation
from IPython import display

In [2]:
year = 2014

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [4]:
cat = col.search(source_id=['NorESM2-LM'], experiment_id=['historical'], table_id=['day'], variable_id=['hus','va'], member_id=['r1i1p1f1'])

In [5]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [6]:
dataset_list = list(dset_dict.keys())

In [7]:
dset = dset_dict[dataset_list[0]]
dset = dset.sel(member_id='r1i1p1f1',time=slice(str(year)+"-01-01", str(year)+"-12-31"))

In [8]:
vas = dset.va
hus = dset.hus
plev = dset.plev
lat_ = hus.lat
lon_ = hus.lon

In [9]:
ivt = compute_ivt(hus,vas,plev)

In [10]:
ivt_ns = ivt.copy()
ivt_ns = xr.where(ivt_ns.lat<0,-ivt_ns,ivt_ns,True) # minus for southern hemisphere (positive toward the pole)
ivt_ns_pos = xr.where(ivt_ns<0,ivt_ns*0,ivt_ns,True) # negative values = not poleward

In [11]:
# pre-computed 98th percentile IVT
q98 = xr.open_dataset('q98.nc')
q98 = q98.rename({'__xarray_dataarray_variable__':'ivt'})

In [18]:
excess = ivt_ns_pos-q98
ar_points = xr.where(excess>0,1,0)

In [ ]:
out_ar = ar_points.copy()
out_ar.ivt.values[:] = out_ar.ivt.values[:]*0.0

res_lon = abs(lon_.values[1]-lon_.values[0])
res_lat = abs(lat_.values[1]-lat_.values[0])

tt=0
for t in out_ar.time:
    df_loc = ar_points.ivt.sel(time=t)
    ll = xr.plot.contour(df_loc,levels=[0,1])
    for item in ll.collections:
        for i in item.get_paths():
            v = i.vertices
            crit = abs(np.max(v[:, 1])-np.min(v[:, 1]))
            if (crit>=20): # AR has to be at least 20 deg lat 
                xx=(v[:, 0]/res_lon).astype(int)
                yy=((v[:, 1]+90)/res_lat).astype(int)
                for (x,y) in zip(xx,yy):
                    out_ar.ivt[y,tt,x]
    tt = tt+1
    plt.close()

In [ ]:
out_ar.to_netcdf(str(year)+'_crit.nc')